<a href="https://colab.research.google.com/github/ccasanoval/RLtests/blob/master/DoomV4c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DoomV4b

RL  = Stable Baseline 3 : PPO

ENV = Gymnasium + VizDoom

URL = https://github.com/AKapich/Reinforcement_Learning_Doom

URL = https://github.com/AKapich/Reinforcement_Learning_Doom/blob/main/MyWayHome/my_way_home_env.py

URL = https://stable-baselines3.readthedocs.io/en/master/

.

Config VizDoom:

http://www.cs.put.poznan.pl/visualdoomai/tutorial.html#config


###.
### ALSO TEST : RLlib

###.
### ALSTO TRY: callbacks

https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/4_callbacks_hyperparameter_tuning.ipynb#scrollTo=adsKMvDkRUn0

###.
### ALSO READ: Algorithm Distilation + In Context RL

https://www.youtube.com/watch?v=BkWLCrLapQo

https://github.com/licong-lin/in-context-rl
    
###.
### ALSO READ:

Automatic hyperparameter optimization

To optimize the parameters of C, we chose the Covariance-Matrix Adaptation Evolution Strategy (CMA-ES)

evolution strategies (ES)

###.
Do androids dream of electric sheep?

https://worldmodels.github.io/

https://github.com/hardmaru/WorldModelsExperiments



In [7]:
!pip install vizdoom
!pip install stable_baselines3
!pip install sb3-contrib

In [8]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#save_path = f"/content/gdrive/MyDrive/Data-PPO-LSTM/{game_name}/"

###################### PARAMETERS #############################################
modelType = "RPPO"     # @param {type:"string"}
modelName = "DoomHome"+modelType # @param {type:"string"}
modelNew = False       # @param {type:"boolean"}
modelTrain = True     # @param {type:"boolean"}



In [9]:
############### GYM ENV == VIZ DOOM ###########################################
from vizdoom import DoomGame, GameVariable
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2

class MyWayHomeGym(Env):
    def __init__(self, scenario, render=True, number_of_actions=3):
        self.game = DoomGame()
        self.game.load_config(f"{scenario}.cfg")

        # self.game.set_mode(Mode.SPECTATOR)  # spectator

        self.game.set_episode_timeout(2100 - 1200)#21 - 6 = 15s

        self.game.add_available_game_variable(GameVariable.POSITION_X)
        self.game.add_available_game_variable(GameVariable.POSITION_Y)
        self.game.add_available_game_variable(GameVariable.POSITION_Z)

        self.pos = None

        self.game.set_window_visible(render)
        self.game.init()

        self.observation_space = Box(
            low=0, high=255, shape=(100, 160, 1), dtype=np.uint8
        )
        self.number_of_actions = number_of_actions
        self.action_space = Discrete(number_of_actions)
        self.wins = 0

        # SAME PLACE PENALTY
        self.pos_history_length = 200
        self.position_history = [None] * self.pos_history_length
        self.i = 0


    def step(self, action):

        actions = np.identity(self.number_of_actions)
        # tics need to be more than 1 to learn (tics = 4)
        tics = 1
        if modelTrain: tics = 4
        reward = self.game.make_action(actions[action], tics = tics)

        dist = 0
        movement_reward = 0
        same_place_penalty = 0
        if self.game.get_state():

            # SAME POSITION PENALTY
            _, pos_x, pos_y, pos_z = self.game.get_state().game_variables
            pos = np.array([pos_x, pos_y, pos_z])
            cur_index = self.i % self.pos_history_length
            self.position_history[cur_index] = pos
            prev_pos = self.position_history[self.pos_history_length - cur_index - 1]
            if np.array_equal(self.position_history[cur_index], prev_pos):
                same_place_penalty = -1
            else:
                same_place_penalty = (
                    -0.5
                    / np.sqrt(
                        np.sum((self.position_history[cur_index] - prev_pos) ** 2)
                    )
                    if prev_pos is not None
                    else 0
                )
            same_place_penalty = max(-1, same_place_penalty)
            self.i += 1

            # MOVEMENT REWARD
            if modelTrain:
                _, pos_x, pos_y, pos_z = self.game.get_state().game_variables
                pos = np.array([pos_x, pos_y, pos_z])
                if self.pos is not None:
                    dist = np.sqrt(np.sum((pos - self.pos) ** 2))
                    movement_reward = dist * 0.005
                    self.pos = pos

            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info": info}
        terminated = self.game.is_episode_finished()

        truncated = (
            self.game.is_player_dead()
            or self.game.is_player_dead()
            or self.game.is_player_dead()
        )

        if reward > 0:
            self.wins = self.wins + 1
            print(f"-************win reward = {reward} wins = {self.wins} *************************")
        elif terminated: print(f"-------------end reward = {reward} / sum = {reward+movement_reward} dist = {dist} movement_reward = {movement_reward} ")
        #elif modelTrain: print(f"-------------end reward = {reward} / sum = {reward+movement_reward} dist = {dist} movement_reward = {movement_reward} ")

        if modelTrain:
            #reward *= 100
            reward += movement_reward
            reward += same_place_penalty
        return state, reward, terminated, truncated, info


    def reset(self, seed=0):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer

        if self.game.get_state():
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            info = 0

        return (self.grayscale(state), {"ammo": info})

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100, 160, 1))
        return state

    def close(self):
        self.game.close()



In [10]:
##################### SB3 : CALLBACK ##########################################
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, verbose=1, name="?"):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.name = name

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = '{}_{}'.format(self.name, self.n_calls)
            self.model.save(model_path)
        return True


In [ ]:
###################### TRAIN == SB3 ###########################################
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib import RecurrentPPO

callback = TrainAndLoggingCallback(check_freq=10000, name=modelName)
env = MyWayHomeGym(render=False, scenario="my_way_home")

if modelNew:
    model = RecurrentPPO("CnnLstmPolicy",
        env,
        verbose=1,
        seed=0,
        #learning_rate=0.0001,
        #n_steps=2048,
    )
else:
    model = RecurrentPPO.load(modelName, env=env)

# TRAIN
if modelTrain:
    model.learn(
        total_timesteps=400000,
        callback=callback,
    )


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
----------------------------
| time/              |     |
|    fps             | 71  |
|    iterations      | 1   |
|    time_elapsed    | 1   |
|    total_timesteps | 128 |
----------------------------
-------------end reward = -0.0004 / sum = -0.0004 dist = 0 movement_reward = 0 
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 225           |
|    ep_rew_mean          | -57.8         |
| time/                   |               |
|    fps                  | 11            |
|    iterations           | 2             |
|    time_elapsed         | 21            |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | 1.1348166e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         |

In [ ]:
###################### TEST == SB3 ########################################
model = RecurrentPPO.load(modelName, env=env)
env = env = MyWayHomeGym(render=True, scenario="my_way_home")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
env.close()

print(f"mean_reward:{mean_reward:.2f}")
print(f"std_reward:{std_reward:.2f}")

